In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import datasets
import numpy as np
import random
from model_structure import get_preprocessing_transforms,BCNN, train_model, evaluate_model

In [2]:
INPUT_SIZE = 224
BATCH_SIZE=32
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
def read_data():
    # Get transforms
    train_transform, val_transform = get_preprocessing_transforms(INPUT_SIZE)

    test_data_1_dir = 'data4model_1/test/'
    test_data_2_dir = 'data4model_2/test/'
    train_data_1_dir = 'data4model_1/train/'
    train_data_2_dir = 'data4model_2/train/'

    # Load data set
    dataset_test_1 = datasets.ImageFolder(test_data_1_dir,transform=val_transform)
    dataset_train_1 = datasets.ImageFolder(train_data_1_dir,transform=train_transform)
    dataset_test_2 = datasets.ImageFolder(test_data_2_dir,transform=val_transform)
    dataset_train_2 = datasets.ImageFolder(train_data_2_dir,transform=train_transform)

    test_loader_1 = DataLoader(dataset_test_1, shuffle=False, batch_size=BATCH_SIZE)
    train_loader_1 = DataLoader(dataset_train_1, shuffle=False, batch_size=BATCH_SIZE)
    test_loader_2 = DataLoader(dataset_test_2, shuffle=False, batch_size=BATCH_SIZE)
    train_loader_2 = DataLoader(dataset_train_2, shuffle=False, batch_size=BATCH_SIZE)

    return {"Model_1:Train": train_loader_1,
            "Model_1:Test": test_loader_1,
            "Model_2:Train": train_loader_2,
            "Model_2:Test": test_loader_2}

In [4]:
model_1_dir = 'best_model_1.pth'
model_2_dir = 'best_model_2.pth'

In [5]:
# Load the model
model_1 = BCNN()
weights = torch.load(model_1_dir, map_location=torch.device(DEVICE))
model_1.load_state_dict(weights)
model_1.eval()

model_2 = BCNN()
weights = torch.load(model_2_dir, map_location=torch.device(DEVICE))
model_2.load_state_dict(weights)
model_2.eval()

C:\Users\hskay\AppData\Local\Temp\ipykernel_3080\2999396719.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(model_1_dir, map_location=torch.device(D

BCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Dropout2d(p=0.25, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.25, inplace=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Dropout2d(p=0.25, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=100352, out_features=512, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=512, out_features=1, bias=True)
    (5): Sigm

In [7]:
dataSets = read_data()
for key in dataSets:
    print(key)
    if "Model_1" in key:
        model = model_1
    else:
        model = model_2
    
    results = evaluate_model(model.to(DEVICE), dataSets[key], DEVICE)
    print("Accuracy: ", results[0])
    print("F1 Score: ", results[1])
    print("confusion Matrix: \n",results[3])
    print("\n")

Model_1:Train
Accuracy:  0.912
F1 Score:  0.9115874079035499
confusion Matrix: 
 [[1375  125]
 [ 139 1361]]


Model_1:Test
Accuracy:  0.875
F1 Score:  0.8743718592964824
confusion Matrix: 
 [[88 12]
 [13 87]]


Model_2:Train
Accuracy:  0.894
F1 Score:  0.8924221921515562
confusion Matrix: 
 [[1363  137]
 [ 181 1319]]


Model_2:Test
Accuracy:  0.905
F1 Score:  0.9090909090909091
confusion Matrix: 
 [[86 14]
 [ 5 95]]


